# Twitter

Is it possible to use the DomainTools list to look for tweets that are potentially spreading misinformation? Fortunately, Twitter's Search does allow for searching by a domain name, try [www.cdc.gov](https://twitter.com/search?q=www.cdc.gov&f=live) for example. However the Search API only returns results for the last week. Given this caveat we can still take a look.

## DomainTools

First we need to load the DoaminTools data:

In [35]:
import pandas

df = pandas.read_csv('data/domaintools/2020-04-05.csv.gz',
    parse_dates=['created'], 
    sep='\t',
    names=['domain', 'created', 'risk']
)
df

,domain,created,risk
0,covid19fashions.com,2020-04-04,99
1,coronaviruscardgame.com,2020-04-04,99
2,covidonline.me,2020-04-04,99
3,covid-19contained.com,2020-04-04,99
4,covidcrime-chicago.com,2020-04-04,99
...,...,...,...
116706,wuhanbj.cn,2020-01-23,70
116707,wuhanpneumonia.org,2020-01-19,70
116708,wuhanfengtai.com,2020-01-13,70
116709,ecovida5dmalaga.com,2020-01-07,70


## Collect Tweets

[twarc](https://github.com/docnow/twarc) is a useful Python library for collecting tweet JSON data while observing their API rate lmits. We have 116,710 domains to search for, so being able to observe the rate limits is important.

[Twitter's Rate Limits](https://developer.twitter.com/en/docs/basics/rate-limits) show that the search API has a limit of 180 requests / 15 minutes for User authentication and 450 requests/ 15 minutes for App authentication. The difference between these is a bit complicated to go into here, but basically User auth is when you want to do something on behalf of a user (say send a tweet), and App auth is when you can do something as a software application. We aren't doing these searches *for* a particular user, and the App authentication will let us performa a lot more queries.

But how long will it take?

In [36]:
reqs_per_hour = 450 * 4
116711 / reqs_per_hour

64.83944444444444

It will take 64 hours! How about if we only look at the domains with the highest risk score?

In [42]:
len(df[df.risk >= 99.0]) / reqs_per_hour

47.946666666666665

Ok, that's still a long time to wait. Since Twitter's API only lets us search tweets for the past 7 days maybe we could get an interesting view by looking for domains that were created during that time? We also might stand a better chance of seeing them before they are deleted by Twitter.

So how long would will this take?

In [55]:
import datetime

a_week_ago = datetime.datetime.now() - datetime.timedelta(days=7)
df['created']

len(df[(df.risk >= 99.0) & (df.created >= a_week_ago)]) / reqs_per_hour

4.051111111111111

4 hours seems manageable! Lets start by looking at those.

In [57]:
latest_domains = df[(df.risk >= 99.0) & (df.created >= a_week_ago)]
latest_domains

,domain,created,risk
0,covid19fashions.com,2020-04-04,99
1,coronaviruscardgame.com,2020-04-04,99
2,covidonline.me,2020-04-04,99
3,covid-19contained.com,2020-04-04,99
4,covidcrime-chicago.com,2020-04-04,99
...,...,...,...
7287,covid19costume.com,2020-03-31,99
7288,covid19testinglocations.co.nz,2020-03-31,99
7289,covidegypt.cf,2020-03-31,99
7290,goawaycovid19.org,2020-03-31,99


In [58]:
import twarc

You can skip this next cell if you've used twarc before. Otherwise you need to run it to tell twarc your API keys.

In [ ]:
t = twarc.Twarc(validate_keys=False)
t.configure()

If you didn't run the cell above you will now need to create your twarc client.

In [59]:
t = twarc.Twarc(app_auth=True)

Now we can search for each donain and write the full tweets as line oriented JSON, where each line is a tweet, serialized as a JSON object. As we go we'll keep track of domains that had some tweet matches.

In [ ]:
import gzip
import json

from collections import Counter

# write the tweets as gzipped json data
out = gzip.open('data/twitter/week-{}.jsonl'.format(a_week_ago.strftime('%Y-%m-%d')), 'wb')

counter = Counter()                
for domain in latest_domains.domain:
    for tweet in t.search(row.domain):
        json.dump(tweet, out)
        counter[domain] += 1
    if counter[domain] > 0:
        sys.stdout.write('{} [{}]'.format(row.domain, counter[domain]))


## Analyze

Ok, that took some time. 